# 1.3.0: Add `Ambience`

In `AudibleLight`, `Ambience` objects represent background audio. They are not "spatialised" (i.e., convolved with IRs) and can be added to 1) challenge a model during training, 2) increase the diversity of generated data, and 3) more accurately represent real-world conditions.

Currently, two forms of background `Ambience` are supported:
- Ambience using an audio file: this will be repeated (tiled) vertically and horizontally to match the length of the `Scene` and the number of channels per microphone added to it.
- Ambience using a noise "type" (e.g., white, pink, Gaussian)

**Note that the volume of an `Ambience` object is set automatically**, according to the `ref_db` value provided to `Scene.__init__`.

## Add "audio" `Ambience`

We can add `Ambience` using an audio file by passing in `filepath=...` to `Scene.add_ambience`.

In [1]:
from audiblelight.ambience import NOISE_MAPPING
from audiblelight.core import Scene
from audiblelight import utils

In [2]:
scene = Scene(
    duration=60,
    sample_rate=44100,
    backend="rlr",
    backend_kwargs=dict(
        mesh=utils.get_project_root() / "tests/test_resources/meshes/Oyens.glb"
    ),
)
scene.add_microphone(microphone_type="ambeovr")

CreateContext: Context created


CreateContext: Context created


In [3]:
scene.add_ambience(filepath=utils.get_project_root() / "tests/test_resources/soundevents/music/000010.mp3", alias="audio1")
amb = scene.get_ambience(alias="audio1")
print(amb.filepath)

/home/huw-cheston/Documents/python_projects/AudibleLight/tests/test_resources/soundevents/music/000010.mp3


Alternatively, we can also provide `bg_path` to `Scene.__init__` to randomly sample audio files from this directory

In [4]:
scene = Scene(
    duration=60,
    sample_rate=44100,
    backend="rlr",
    backend_kwargs=dict(
        mesh=utils.get_project_root() / "tests/test_resources/meshes/Oyens.glb"
    ),
    bg_path=utils.get_project_root() / "tests/test_resources/soundevents/music"
)
scene.add_microphone(microphone_type="ambeovr")

CreateContext: Context created
CreateContext: Context created


In [5]:
scene.add_ambience(filepath=None, alias="audio2")
amb2 = scene.get_ambience("audio2")
print(amb2.filepath)

/home/huw-cheston/Documents/python_projects/AudibleLight/tests/test_resources/soundevents/music/001666.mp3


When we load the ambience, we shall see that it has automatically been tiled horizontally to match the duration and sample rate of the scene, and vertically to match the channel layout of our microphone:

In [6]:
out = amb2.load_ambience(ignore_cache=True)
print(out.shape)

2025-10-07 16:58:33.440 | WARNING  | audiblelight.ambience:load_ambience:188 - Passed audio has 2 channels, but expected 4 channels. A random mono channel will be chosen from the audio.


(4, 2646000)


In [7]:
print(scene.sample_rate * scene.duration)
print(scene.get_microphone("mic000").n_capsules)

2646000.0
4


## Add "noise" `Ambience`

We can also add `Ambience` according to a particular type of (coloured) noise. This can be represented as either a string (e.g., "white", "Gaussian") or a numeric exponent. In general terms, higher exponents cause more energy in high frequency parts of the audio spectrum.

In [8]:
scene.clear_ambience()
scene.add_ambience(noise="white", alias="whitenoise")

# Equivalent:
# scene.add_ambience(noise=0.0, alias="beta2noise")

To see the types of noise that are available:

In [9]:
print(NOISE_MAPPING.keys())

dict_keys(['pink', 'brown', 'red', 'blue', 'white', 'violet'])


Again, note that the duration of the `Ambience` audio is set according to the `Scene` and microphone objects added to it:

In [10]:
ambwn = scene.get_ambience("whitenoise")
ambwn_aud = ambwn.load_ambience(ignore_cache=True)
print(ambwn_aud.shape)

(4, 2646000)


Audio associated with any `Ambience` objects added to a `Scene` is automatically rendered whenever `Scene.generate` is called.